Import the pandas library and open the HESA database (NOTE: the data path is correct if the same hierarchy as the GitHub page is followed)

In [23]:
import pandas
# Edit the years to match the datasets that should be merged 
year_hesa = '2019-20'
year_other_data = 2019

data_path = f'../data/hesa_cleaned/fulltime_undergrads_{year_hesa}.csv'
hesa =  pandas.read_csv(data_path, index_col=0)

gdp_path = '../data/non_hesa_factors/gdp_per_capita.csv'
gdp =  pandas.read_csv(gdp_path, index_col=0)

gini_path = '../data/non_hesa_factors/gini_index.xls'
gini =  pandas.read_excel(gini_path, index_col=0, skiprows=3)

tert_exp_path = '../data/non_hesa_factors/tertiary_education_expenditure.xls'
tert_exp =  pandas.read_excel(tert_exp_path, index_col=0, skiprows=3)

tert_enr_path = '../data/non_hesa_factors/tertiary_education_enrollment.xls'
tert_enr =  pandas.read_excel(tert_enr_path, index_col=0, skiprows=3)

pop_path = '../data/non_hesa_factors/population.csv'
pop =  pandas.read_csv(pop_path, index_col=0, skiprows=4)

hdi_path = '../data/non_hesa_factors/hdi.csv'
hdi = pandas.read_csv(hdi_path, index_col=1, skiprows=5)
hdi.index = hdi.index.str.strip() # Certain rows in the country column contain a whitespace, which must be removed to compare with other datasets 


A quick overview of two databases:

In [31]:
tert_enr.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.725639,37.761131,16.195829,16.40542,15.27177,15.59921,NaN,NaN,NaN,NaN
Africa Eastern and Southern,AFE,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Afghanistan,AFG,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.622260,NaN,NaN,8.23068,NaN,NaN,NaN,9.68642,NaN,10.56252
Africa Western and Central,AFW,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,AGO,"School enrollment, tertiary (% gross)",SE.TER.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.130300,NaN,8.838330,NaN,8.40094,9.33626,NaN,NaN,NaN,NaN


In [32]:
gdp.head()

,Code,Year,"GDP per capita, PPP (constant 2017 international $)"
Entity,,,
Afghanistan,AFG,2002,1189.784668
Afghanistan,AFG,2003,1235.810063
Afghanistan,AFG,2004,1200.278013
Afghanistan,AFG,2005,1286.793659
Afghanistan,AFG,2006,1315.789117


Unlike the other databases, years in GDP are sorted by row. The following code selects the yearly data and converts it into a single column for compatibility with the other datasets:

In [16]:
gdp_step_1 = gdp.loc[gdp['Year'] == year_other_data]
gdp_definitive = gdp_step_1[['GDP per capita, PPP (constant 2017 international $)']]
gdp_definitive = gdp_definitive.rename(columns={'GDP per capita, PPP (constant 2017 international $)': 'GDP per Capita'})
gdp_definitive.head()

,GDP per Capita
Entity,
Afghanistan,2065.036235
Africa Eastern and Southern,3568.597017
Africa Western and Central,4142.398382
Albania,13671.488422
Algeria,11510.557088


The single year is also selected in other datasets:

In [17]:
tert_enr_definitive = tert_enr[[f'{year_other_data}']]
tert_enr_definitive = tert_enr_definitive.rename(columns={f'{year_other_data}': 'Tertiary Education Enrollment'})

tert_exp_definitive = tert_exp[[f'{year_other_data}']]
tert_exp_definitive = tert_exp_definitive.rename(columns={f'{year_other_data}': 'Tertiary Expenditure'})

gini_definitive = gini[[f'{year_other_data}']]
gini_definitive = gini_definitive.rename(columns={f'{year_other_data}': 'GINI'})

pop_definitive = pop[[f'{year_other_data}']]
pop_definitive = pop_definitive.rename(columns={f'{year_other_data}': 'Population'})

hdi_definitive = hdi[[f'{year_other_data}']]
hdi_definitive = hdi_definitive.rename(columns={f'{year_other_data}': 'HDI'})

One by one, all datasets are merged together to form the complete dataset for the year:

In [18]:
hesa_gdp_joined = hesa.merge(gdp_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_joined = hesa_gdp_joined.merge(tert_enr_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_tertexp_joined = hesa_gdp_tertenr_joined.merge(tert_exp_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_tertexp_gini_joined = hesa_gdp_tertenr_tertexp_joined.merge(gini_definitive, left_index=True, right_index=True, how='left')

hesa_gdp_tertenr_tertexp_gini_joined = hesa_gdp_tertenr_tertexp_gini_joined.rename(columns={'Number': 'Number of Undergraduate Students'})
hesa_gdp_tertenr_tertexp_gini_pop_joined = hesa_gdp_tertenr_tertexp_gini_joined.merge(pop_definitive, left_index=True, right_index=True, how='left')
hesa_gdp_tertenr_tertexp_gini_pop_hdi_joined = hesa_gdp_tertenr_tertexp_gini_pop_joined.merge(hdi_definitive, left_index=True, right_index=True, how='left')

hesa_gdp_tertenr_tertexp_gini_pop_hdi_joined.head()

,Number of Undergraduate Students,GDP per Capita,Tertiary Education Enrollment,Tertiary Expenditure,GINI,Population,HDI
Country of domicile,,,,,,,
Åland Islands,0,NaN,NaN,NaN,NaN,NaN,NaN
Austria,50,55833.315686,86.475533,NaN,NaN,8879920.0,0.922
Belgium,80,51742.726383,80.138168,NaN,NaN,11488980.0,0.931
Bulgaria,95,23191.574585,73.379173,NaN,NaN,6975761.0,0.816
Canary Islands,0,NaN,NaN,NaN,NaN,NaN,NaN


The database is exported to a .csv file:

In [19]:
hesa_gdp_tertenr_tertexp_gini_pop_joined.to_csv(f'../data/all_merged_by_year/all_databases_merged_{year_other_data}.csv', index=True)